In [1]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pyemu
import flopy

In [2]:
# truth dir
truth_d = os.path.join('..','..', 'models', 'freyberg_mf6_truth')

# remove existing folder
if os.path.exists(truth_d):
    shutil.rmtree(truth_d)
# make truth dir
os.mkdir(truth_d)

In [3]:
# pest setup template folder
t_d = os.path.join('..', '..', 'models', 'freyberg_pest_setup')
pst = pyemu.Pst(os.path.join(t_d, 'freyberg_mf6.pst'))

# choose realisation; this one gives extreme-ish headwater predictions
real=61
pst.parrep(parfile=os.path.join(t_d, 'prior.jcb'), real_name=61, binary_ens_file=True)
pst.write_input_files(pst_path=t_d)

# run forward run so that parameter input files are updated; 
pyemu.os_utils.run('python forward_run.py', cwd=t_d)

updating parval1 using realization:'61' from ensemble file ..\..\models\freyberg_pest_setup\prior.jcb
parrep: updating noptmax to 0


In [4]:
# load, rewrite and run simulation
sim = flopy.mf6.MFSimulation.load(sim_ws=t_d, verbosity_level=0)

sim.set_sim_path(truth_d)
sim.set_all_data_external(check_data=True)
sim.write_simulation()

# run mf6 so that model output files are avilable
pyemu.os_utils.run('mf6', cwd=truth_d)

In [5]:
# rename modle otpu files because of silly design decisions a while back
for f in [f for f in os.listdir(truth_d) if f.endswith('.csv')]:
    os.rename(os.path.join(truth_d, f), os.path.join(truth_d, f.split('.')[0]+'.meas.csv'))